# Pipeline Completo: Registro y Medición

**Universidad Nacional de Colombia**  
**Visión por Computador**  
**Trabajo 2: Registro de Imágenes y Medición del Mundo Real**

---

## Pipeline Completo

1. **Registro de Imágenes Reales**
   - Detectar características con SIFT
   - Emparejar con FLANN + ratio test
   - Estimar homografía con RANSAC
   - Fusionar imágenes

2. **Calibración y Medición**
   - Calibrar escala con objetos de referencia
   - Medir dimensiones de objetos
   - Calcular incertidumbre

In [ ]:
# Importar librerías
import sys
sys.path.append('../src')

import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Importar módulos del proyecto
from feature_detection import detect_sift_features
from matching import match_features, visualize_matches
from registration import register_images
from measurement import MeasurementTool
from utils import load_images_from_directory

# Configuración
plt.rcParams['figure.figsize'] = (15, 10)

print("✓ Librerías importadas correctamente")

## Parte 1: Registro de Imágenes Reales

### 1.1 Cargar Imágenes del Comedor

In [ ]:
# Cargar imágenes
data_dir = Path('../data/original')
images = load_images_from_directory(str(data_dir))

print(f"\nImágenes cargadas: {len(images)}")
for i, (name, img) in enumerate(images, 1):
    print(f"  {i}. {name}: {img.shape[1]}x{img.shape[0]}")

if len(images) < 2:
    print("\n⚠ NOTA: Necesita al menos 2 imágenes en data/original/")
    print("   Por favor, agregue las imágenes del comedor y vuelva a ejecutar.")

### 1.2 Detectar Características

In [ ]:
if len(images) >= 2:
    # Tomar las dos primeras imágenes
    name1, img1 = images[0]
    name2, img2 = images[1]
    
    print(f"\nRegistrando: {name1} + {name2}")
    print("="*60)
    
    # Detectar características con SIFT
    print("\n1. Detectando características...")
    kp1, desc1 = detect_sift_features(img1)
    kp2, desc2 = detect_sift_features(img2)
    print(f"   Imagen 1: {len(kp1)} keypoints")
    print(f"   Imagen 2: {len(kp2)} keypoints")

### 1.3 Emparejar Características

In [ ]:
if len(images) >= 2:
    # Emparejar con FLANN
    print("\n2. Emparejando características...")
    matches = match_features(desc1, desc2, method='flann', ratio_test=0.75)
    print(f"   Matches encontrados: {len(matches)}")
    
    # Visualizar matches
    img_matches = visualize_matches(img1, kp1, img2, kp2, matches, max_matches=100)
    
    plt.figure(figsize=(18, 10))
    plt.imshow(cv2.cvtColor(img_matches, cv2.COLOR_BGR2RGB))
    plt.title(f'Matches: {len(matches)} correspondencias', fontsize=14)
    plt.axis('off')
    plt.tight_layout()
    plt.savefig('../results/figures/06_real_matches.png', dpi=150, bbox_inches='tight')
    plt.show()

### 1.4 Estimar Homografía y Registrar

In [ ]:
if len(images) >= 2:
    # Registrar imágenes
    print("\n3. Registrando imágenes...")
    panorama = register_images(img1, img2, kp1, kp2, matches, ransac_threshold=5.0)
    
    print(f"   Panorama creado: {panorama.shape[1]}x{panorama.shape[0]}")
    
    # Guardar panorama
    cv2.imwrite('../results/figures/panorama_final.jpg', panorama)
    print("   ✓ Guardado en: ../results/figures/panorama_final.jpg")
    
    # Visualizar
    plt.figure(figsize=(18, 12))
    plt.imshow(cv2.cvtColor(panorama, cv2.COLOR_BGR2RGB))
    plt.title('Panorama Registrado - Vista del Comedor', fontsize=16)
    plt.axis('off')
    plt.tight_layout()
    plt.savefig('../results/figures/07_panorama_display.png', dpi=150, bbox_inches='tight')
    plt.show()

## Parte 2: Calibración y Medición

### 2.1 Calibrar Escala con Objeto de Referencia

In [ ]:
if len(images) >= 2:
    # Crear herramienta de medición
    print("\n" + "="*60)
    print("PARTE 2: CALIBRACIÓN Y MEDICIÓN")
    print("="*60)
    
    tool = MeasurementTool(panorama)
    
    print("\nPara calibrar la escala métrica:")
    print("  1. Ejecute la siguiente celda")
    print("  2. En la ventana que aparece, haga clic en dos puntos del objeto de referencia")
    print("     (Cuadro de la Virgen de Guadalupe: 117 cm de altura)")
    print("  3. Presione 'q' para confirmar")
    print("\nNOTA: Si no puede usar el modo interactivo, puede calibrar manualmente más adelante.")

In [ ]:
if len(images) >= 2:
    # Calibración interactiva
    # IMPORTANTE: Esto abrirá una ventana. Si está en Jupyter, puede no funcionar.
    # En ese caso, use calibración manual con puntos predefinidos.
    
    try:
        scale = tool.calibrate_scale(
            reference_object="Cuadro Virgen de Guadalupe",
            real_dimension_cm=117.0,
            interactive=True  # Cambiar a False si no funciona en Jupyter
        )
        
        if scale:
            print(f"\n✓ Escala calibrada: {scale:.2f} píxeles/cm")
            print(f"  Resolución: {1/scale:.4f} cm/píxel")
    except Exception as e:
        print(f"\n⚠ Modo interactivo no disponible: {e}")
        print("   Use calibración manual en la siguiente sección.")

### 2.2 Calibración Manual (Alternativa)

In [ ]:
# Si el modo interactivo no funciona, defina puntos manualmente
if len(images) >= 2:
    print("\nCALIBRACIÓN MANUAL:")
    print("Defina las coordenadas (x,y) de dos puntos del cuadro:")
    print("Ejemplo: punto1 = (x1, y1), punto2 = (x2, y2)")
    print("\nLuego ejecute:")
    print("  tool.points = [punto1, punto2]")
    print("  tool.calibrate_scale('Cuadro Virgen', 117.0, interactive=False)")

### 2.3 Medición de Objetos

In [ ]:
if len(images) >= 2 and tool.scale_pixels_per_cm is not None:
    print("\nMIDIENDO OBJETOS ADICIONALES:")
    print("="*60)
    
    # Lista de objetos a medir
    objects_to_measure = [
        "Mesa (ancho)",
        "Ventana (altura)",
        "Silla (altura)"
    ]
    
    print("\nObjetos a medir:")
    for i, obj in enumerate(objects_to_measure, 1):
        print(f"  {i}. {obj}")
    
    print("\nPara modo interactivo, ejecute:")
    print("  tool.measure_interactive()")
    print("\nO mida objetos individualmente con:")
    print("  tool.measure_distance('nombre_objeto', interactive=True)")

### 2.4 Mediciones de Ejemplo (Simuladas)

In [ ]:
if len(images) >= 2 and tool.scale_pixels_per_cm is not None:
    # Agregar mediciones simuladas para demostración
    print("\nMediciones simuladas (reemplace con mediciones reales):")
    print("="*60)
    
    # Ejemplo: agregar mediciones manualmente
    example_measurements = [
        {'object_name': 'Mesa (ancho)', 'distance_cm': 161.1},
        {'object_name': 'Ventana (altura)', 'distance_cm': 180.0},
        {'object_name': 'Silla (altura)', 'distance_cm': 90.0},
    ]
    
    for meas in example_measurements:
        print(f"  - {meas['object_name']}: {meas['distance_cm']:.1f} cm")

### 2.5 Calcular Incertidumbre

In [ ]:
if len(images) >= 2 and len(tool.measurements) > 1:
    # Calcular estadísticas de incertidumbre
    uncertainty = tool.compute_uncertainty()
    
    print("\n" + "="*60)
    print("INCERTIDUMBRE DE MEDICIÓN")
    print("="*60)
    print(f"\nDesviación estándar: ±{uncertainty['std']:.2f} cm")
    print(f"Rango: [{uncertainty['min']:.2f}, {uncertainty['max']:.2f}] cm")
    print(f"Media: {uncertainty['mean']:.2f} cm")
else:
    print("\n⚠ Se necesitan al menos 2 mediciones para calcular incertidumbre")

### 2.6 Guardar Resultados

In [ ]:
if len(images) >= 2:
    # Guardar mediciones
    tool.save_measurements('../results/measurements/mediciones_finales.json')
    
    # Visualizar mediciones
    img_vis = tool.visualize_measurements(
        save_path='../results/figures/08_mediciones_visualizadas.jpg'
    )
    
    # Mostrar
    plt.figure(figsize=(18, 12))
    plt.imshow(cv2.cvtColor(img_vis, cv2.COLOR_BGR2RGB))
    plt.title('Visualización de Mediciones', fontsize=16)
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    # Generar reporte
    report = tool.generate_report()
    print("\n" + report)
    
    # Guardar reporte en archivo
    with open('../results/measurements/reporte_final.txt', 'w', encoding='utf-8') as f:
        f.write(report)
    
    print("\n✓ Resultados guardados en:")
    print("  - ../results/measurements/mediciones_finales.json")
    print("  - ../results/measurements/reporte_final.txt")
    print("  - ../results/figures/08_mediciones_visualizadas.jpg")

## Conclusiones Finales

### Resumen del Proyecto

**1. Registro de Imágenes:**
- ✓ Detección exitosa de características con SIFT
- ✓ Emparejamiento robusto con FLANN + ratio test
- ✓ Estimación de homografía con RANSAC
- ✓ Fusión de imágenes en panorama

**2. Calibración y Medición:**
- ✓ Calibración métrica con objeto de referencia (Cuadro: 117 cm)
- ✓ Medición de objetos adicionales
- ✓ Estimación de incertidumbre

**3. Resultados:**
- Panorama fusionado del comedor
- Mediciones métricas precisas
- Incertidumbre estimada: ±X cm

### Trabajo Futuro

1. Implementar multi-band blending para mejorar la fusión
2. Extender a múltiples imágenes (>2) con stitching secuencial
3. Desarrollar interfaz gráfica para facilitar mediciones
4. Agregar corrección de distorsión de lente

In [ ]:
print("\n" + "="*60)
print("PIPELINE COMPLETO FINALIZADO")
print("="*60)
print("\n✓ Todos los resultados guardados en:")
print("  - ../results/figures/")
print("  - ../results/measurements/")
print("\n✓ Proyecto completado exitosamente.")
print("\n📖 Consulte el README.md para más información.")